# Labo 5 Modèle word2vec et ses applications

## 1. Tester et évaluer un modèle déjà entraîné sur Google News

Veuillez télécharger le modèle word2vec pré-entraîné sur le corpus Google News en écrivant :
```python
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")
```
ce qui téléchargera le fichier la première fois.
Après avoir téléchargé le modèle, vous pourrez l’utiliser ainsi (dans le dossier gensim-data) :
```python
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)
```

In [4]:
# Takes ~30min to download. Only execute once
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

In [5]:
# Model is then located at home directory, adjust as needed.
import os
file_path = os.path.join(os.path.expanduser("~"), "gensim-data", "word2vec-google-news-300", "word2vec-google-news-300.gz")

In [6]:
# Takes ~1min
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load_word2vec_format(file_path, binary=True)

### a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ?

In [7]:
import psutil
import os

# Get the current process ID
process = psutil.Process(os.getpid())

# Get memory information in bytes and convert to MB
memory_info = process.memory_info()
memory_usage_mb = memory_info.rss / 1024 / 1024  # Convert to MB

print(f"Memory usage of the notebook process: {memory_usage_mb:.2f} MB")
print(f"Word2vec model size: {w2v_vectors.vectors.nbytes / 1024 / 1024:.2f} MB")

Memory usage of the notebook process: 1417.81 MB
Word2vec model size: 3433.23 MB


### b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?

In [8]:
print(f"Vector dimension: {w2v_vectors.vector_size}")

Vector dimension: 300


### c. Quelle est la taille du vocabulaire connu du modèle ? Veuillez afficher cinq mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas.


In [9]:
# Get vocabulary size
vocab_size = len(w2v_vectors)
print(f"Vocabulary size: {vocab_size}")

# Five words in vocabulary
words_in_vocab = ['baste', 'spleen', 'maudlin', 'saudade', 'aa']
print("\nFive words in vocabulary:")
for word in words_in_vocab:
    if word in w2v_vectors:
        print(f"'{word}' is in vocabulary")
    else:
        print(f"'{word}' is NOT in vocabulary")

# Two words not in vocabulary
words_not_in_vocab = ['enneahedron', 'petrichor']
print("\nTwo words not in vocabulary:")
for word in words_not_in_vocab:
    if word in w2v_vectors:
        print(f"'{word}' is in vocabulary")
    else:
        print(f"'{word}' is NOT in vocabulary")

Vocabulary size: 3000000

Five words in vocabulary:
'baste' is in vocabulary
'spleen' is in vocabulary
'maudlin' is in vocabulary
'saudade' is in vocabulary
'aa' is in vocabulary

Two words not in vocabulary:
'enneahedron' is NOT in vocabulary
'petrichor' is NOT in vocabulary


### d. Quelle est la similarité entre les mots rabbit et carrot ? Veuillez rappeler comment on mesure les similarités entre deux mots grâce à leurs vecteurs.

In [10]:
similarity = w2v_vectors.similarity('rabbit', 'carrot')
print(f"Similarity between 'rabbit' and 'carrot': {similarity:.4f}")

Similarity between 'rabbit' and 'carrot': 0.3631


> La distance cosinus mesure la différence entre deux vecteurs en évaluant l'angle qui les sépare. Pour la calculer, on détermine la similitude cosinus, qui est le rapport entre le produit de ces vecteurs et le produit de leurs longueurs. Ici, la distance est de 0.3631. Puisque le cosinus a des valeurs entre -1 et 1 (1 signifie que les deux vecteurs sont identiques et -1 signifie qu'ils sont opposés), on peut dire ici que les deux mots sont assez proches mais pas énormément.

### e. Considérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la similarité entre les deux mots. Veuillez indiquer si les similarités obtenues correspondent à vos intuitions sur la proximité des sens des mots.

In [11]:
pairs = [
    ('king', 'queen'),
    ('rabbit', 'bunny'),
    ('rabbit', 'hare'),
    ('plane', 'tower'),
    ('duc', 'duchesse')
]

for pair in pairs:
    word1, word2 = pair
    if word1 in w2v_vectors and word2 in w2v_vectors:
        similarity = w2v_vectors.similarity(word1, word2)
        print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")
    else:
        print(f"One or both words in the pair '{pair}' are not in the vocabulary.")

Similarity between 'king' and 'queen': 0.6511
Similarity between 'rabbit' and 'bunny': 0.6440
Similarity between 'rabbit' and 'hare': 0.6115
Similarity between 'plane' and 'tower': 0.2549
Similarity between 'duc' and 'duchesse': 0.3245


> Les similarités cosinus calculées pour les paires de mots correspondent en grande partie à l'intuition que nous avons de leur proximité :

 - 'King' et 'Queen' (0.6511) : Une similarité élevée qui reflète leur relation étroite en tant que termes royaux liés par leur rôle.

 - 'Rabbit' et 'Bunny' (0.6440) : Une similarité élevée qui est logique, car ce sont des termes souvent utilisés pour désigner le même animal.

 - 'Rabbit' et 'Hare' (0.6115) : Une similarité notable, car ce sont deux animaux très similaires même s'ils représentent des espèces différentes.

 - 'Plane' et 'Tower' (0.2549) : Une similarité faible, ce qui est conforme à notre intuition, car ce sont deux concepts très différents, l’un étant un moyen de transport et l’autre une structure.

 - 'Duc' et 'Duchesse' (0.3245) : Une similarité relativement faible, ce qui est surprenant, car ces deux termes sont des titres de noblesse étroitement liés. Cela pourrait indiquer une faible représentation de ces termes dans le modèle utilisé, ou une distinction plus marquée entre les genres.

Dans l'ensemble, les valeurs correspondent globalement à nos intuitions, bien que la similarité entre 'Duc' et 'Duchesse' semble sous-estimée.

### f. Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ? Comment expliquez-vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

In [12]:
similarity = w2v_vectors.similarity('black', 'white')
print(f"Similarity: {similarity:.4f}")

Similarity: 0.8092


> La similarité élevée (0.8092) entre 'black' et 'white' dans Word2Vec s'explique par le fait que ces deux mots apparaissent souvent dans des contextes similaires, comme des descriptions de couleurs, des oppositions dans des expressions ou des concepts. Word2Vec capture les relations contextuelles plutôt que les significations, ce qui entraîne une proximité vectorielle même pour des antonymes. Cela montre que le modèle est efficace pour identifier les liens contextuels entre les mots, mais qu'il ne différencie pas les synonymes des antonymes, ce qui peut être un inconvénient dans certaines applications qui nécessitent une compréhension des sens opposés.

### g. En vous aidant de la documentation de Gensim sur KeyedVectors, obtenez les scores du modèle word2vec sur les données de test WordSimilarity-353. Veuillez rappeler en 1-2 phrases comment les différents scores sont calculés.

In [13]:
from gensim.test.utils import datapath

word_sim_353_path = datapath('wordsim353.tsv')
result = w2v_vectors.evaluate_word_pairs(word_sim_353_path)

print(f"Pearson correlation coefficient: {result[0][0]:.4f}")
print(f"Pearson correlation p-value: {result[0][1]:.4f}")
print(f"Spearman rank correlation coefficient: {result[1][0]:.4f}")
print(f"Spearman rank correlation p-value: {result[1][1]:.4f}")
print(f"Ratio of pairs with OOV words: {result[2]:.2%}")

Pearson correlation coefficient: 0.6239
Pearson correlation p-value: 0.0000
Spearman rank correlation coefficient: 0.6589
Spearman rank correlation p-value: 0.0000
Ratio of pairs with OOV words: 0.00%


> Les scores de similarité entre les paires de mots sont évalués avec deux méthodes : le coefficient de corrélation de Pearson, qui mesure la corrélation linéaire entre les similarités prédites par le modèle et les similarités humaines, et le coefficient de corrélation de Spearman, qui évalue la cohérence des rangs (classements) des paires de mots. Les p-valeurs associées indiquent la significativité statistique de ces corrélations, tandis que le ratio de paires avec des mots hors vocabulaire (OOV) montre la proportion de paires que le modèle n'a pas pu évaluer.

### h. En vous aidant de la documentation, calculez le score du modèle word2vec sur les données questions-words.txt. Attention, cette évaluation prend une dizaine de minutes, donc il vaut mieux commencer par tester avec un fragment de ce fichier (copier/coller les 100 premières analogies). Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [14]:
from gensim.test.utils import datapath
import time

start_time = time.time()

# Load questions-words.txt from gensim (path is : /.venv/lib/python3.12/site-packages/gensim/test/test_data/questions-words.txt
analogies_path = datapath('questions-words.txt')
#analogies_path = "./questions-words-smaller.txt"

# Example of result : (0.0, [ {'section': 'capital-common-countries', 'correct': [], 'incorrect':[...]} ] )
result = w2v_vectors.evaluate_word_analogies(analogies_path)

elapsed_time = time.time() - start_time

print(f"Total accuracy: {result[0]:.4f}")
print(f"Number of sections: {len(result[1])}")
print(f"Evaluation time: {elapsed_time:.2f} seconds")

print("\nSection-wise results:")
for res in result[1]:
    correct_nbr = len(res["correct"])
    total_nbr = correct_nbr + len(res["incorrect"])
    print(f"Section '{res["section"]}': {correct_nbr/total_nbr:.4f} accuracy ({correct_nbr}/{total_nbr})")

Total accuracy: 0.7401
Number of sections: 15
Evaluation time: 156.10 seconds

Section-wise results:
Section 'capital-common-countries': 0.8320 accuracy (421/506)
Section 'capital-world': 0.8132 accuracy (3552/4368)
Section 'currency': 0.2847 accuracy (230/808)
Section 'city-in-state': 0.7211 accuracy (1779/2467)
Section 'family': 0.8617 accuracy (436/506)
Section 'gram1-adjective-to-adverb': 0.2923 accuracy (290/992)
Section 'gram2-opposite': 0.4347 accuracy (353/812)
Section 'gram3-comparative': 0.9129 accuracy (1216/1332)
Section 'gram4-superlative': 0.8797 accuracy (987/1122)
Section 'gram5-present-participle': 0.7850 accuracy (829/1056)
Section 'gram6-nationality-adjective': 0.9018 accuracy (1442/1599)
Section 'gram7-past-tense': 0.6538 accuracy (1020/1560)
Section 'gram8-plural': 0.8701 accuracy (1159/1332)
Section 'gram9-plural-verbs': 0.6816 accuracy (593/870)
Section 'Total accuracy': 0.7401 accuracy (14307/19330)


> Le score du modèle Word2Vec sur les analogies est calculé en mesurant la proportion de questions correctement résolues parmi les analogies testées. Chaque analogie suit le format "A est à B ce que C est à ?", et le modèle tente de deviner le mot D. Le score mesure la capacité du modèle à capturer des relations sémantiques entre les mots (comme les relations entre capitales et pays, comparatifs, ou opposés).

## 2. Entraîner deux nouveaux modèles word2vec à partir de deux corpus

### a. En utilisant gensim.downloader (voir question 1) récupérez le corpus qui contient les 108 premiers caractères de Wikipédia (en anglais) avec la commande : corpus = api.load('text8'). Combien de phrases et de mots (tokens) possède ce corpus ?

In [15]:
corpus = api.load('text8')
sentences = list(corpus)

num_sentences = len(sentences)
num_tokens = sum(len(sentence) for sentence in sentences)

print(f"Nombre de phrases : {num_sentences}")
print(f"Nombre de mots (tokens) : {num_tokens}")

Nombre de phrases : 1701
Nombre de mots (tokens) : 17005207


### b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus (voir la documentation de Word2vec). Si nécessaire, procédez progressivement, en commençant par utiliser 1% du corpus, puis 10%, etc., pour contrôler le temps nécessaire.
- Veuillez indiquer la dimension choisie pour le embedding de ce nouveau modèle.
- Combien de temps prend l’entraînement sur le corpus total ?
- Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [16]:
from gensim.models import Word2Vec

In [17]:
embedding_dim = 100

corpus_sizes = [0.01, 0.1, 1.0]
for size in corpus_sizes:
    print(f"\nEntraînement avec {int(size * 100)}% du corpus :")
    subset = sentences[:int(len(sentences) * size)]

    start_time = time.time()
    model = Word2Vec(sentences=subset, vector_size=embedding_dim, window=5, min_count=5, sg=1, epochs=10)
    training_time = time.time() - start_time
    
    model_file = f"text8_word2vec_{int(size * 100)}.model"
    model.save(model_file)
    
    model_size_mb = os.path.getsize(model_file) / (1024 * 1024)
    
    print(f"Temps d'entraînement : {training_time:.2f} secondes")
    print(f"Taille du modèle : {model_size_mb:.2f} Mo")


Entraînement avec 1% du corpus :
Temps d'entraînement : 1.54 secondes
Taille du modèle : 3.18 Mo

Entraînement avec 10% du corpus :
Temps d'entraînement : 17.54 secondes
Taille du modèle : 15.41 Mo

Entraînement avec 100% du corpus :
Temps d'entraînement : 223.43 secondes
Taille du modèle : 56.47 Mo


### c. Mesurez la qualité de ce modèle comme en (1g) et (1h). Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle est selon vous la raison de la différence ?

In [18]:
wordsim353_sim = model.wv.evaluate_word_pairs(datapath("wordsim353.tsv"))
print(f"Pearson correlation: {wordsim353_sim[0][0]:.4f} (p-value: {wordsim353_sim[0][1]})")
print(f"Spearman correlation: {wordsim353_sim[1][0]:.4f} (p-value: {wordsim353_sim[1][1]})")
print(f"OOV ratio: {wordsim353_sim[2]:.4f}")

analogy_scores = model.wv.evaluate_word_analogies(datapath('questions-words.txt'))
print("Score :", analogy_scores[0])

Pearson correlation: 0.6776 (p-value: 1.6706526799189313e-48)
Spearman correlation: 0.6906 (p-value: 4.667657820285376e-51)
OOV ratio: 0.5666
Score : 0.33280978291355806


> Le modèle présente une corrélation de Pearson (0,6776) et de Spearman (0,6906) légèrement meilleures que le modèle Google News sur WordSimilarity-353, mais un score d'analogies (0,3328) bien inférieur. Cela s'explique par le fait que votre modèle couvre moins de vocabulaire (OOV de 56,66%), ce qui limite sa performance sur les analogies. Le modèle Google News est entraîné sur un très grand corpus de textes variés, il offre donc une couverture de vocabulaire bien plus large et des représentations vectorielles plus robustes, ce qui améliore sa capacité à résoudre les analogies.

### d. Téléchargez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters fourni par l’enseignant et appelé wikipedia_augmented.zip (à décompresser en un fichier ‘.dat’ de 413 Mo). Entraînez un nouveau modèle word2vec sur ce corpus, en précisant la dimension choisie pour les embeddings.
- Utilisez la classe Text8Corpus() pour charger ce corpus, ce qui fera automatiquement la
tokenisation et la segmentation en phrases.
- Combien de temps prend l’entraînement ?
- Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [22]:
from gensim.models.word2vec import Text8Corpus

In [24]:
corpus_path = "wikipedia_augmented.dat"
corpus = Text8Corpus(corpus_path)

embedding_dim = 100

start_time = time.time()
new_model = Word2Vec(sentences=corpus, vector_size=embedding_dim, window=5, min_count=5, sg=1, epochs=10)
training_time = time.time() - start_time

model_file = "wikipedia_augmented_word2vec.model"
new_model.save(model_file)

model_size_mb = os.path.getsize(model_file) / (1024 * 1024)

print(f"Temps d'entraînement : {training_time:.2f} secondes")
print(f"Taille du modèle : {model_size_mb:.2f} Mo")

Temps d'entraînement : 1040.42 secondes
Taille du modèle : 3.71 Mo


> Le temps d'entraînement est de 17 minutes et demi à peu près et la taille est de 3,71 Mo

### e. Testez ce modèle comme en (1g) et (1h). Est-il meilleur que le précédent ?

In [25]:
wordsim353_sim = new_model.wv.evaluate_word_pairs(datapath("wordsim353.tsv"))
print(f"Pearson correlation: {wordsim353_sim[0][0]:.4f} (p-value: {wordsim353_sim[0][1]})")
print(f"Spearman correlation: {wordsim353_sim[1][0]:.4f} (p-value: {wordsim353_sim[1][1]})")
print(f"OOV ratio: {wordsim353_sim[2]:.4f}")

analogy_scores = new_model.wv.evaluate_word_analogies(datapath('questions-words.txt'))
print("Score :", analogy_scores[0])

Pearson correlation: 0.5434 (p-value: 1.6618050648021703e-28)
Spearman correlation: 0.5427 (p-value: 1.9899612315550654e-28)
OOV ratio: 0.0000
Score : 0.35854905373069396


| Modèle                  | Pearson Correlation | Spearman Correlation | OOV Ratio |
| ----------------------- | ------------------- | -------------------- | --------- |
| Modèle 1 (w2v\_vectors) | 0.6239 (p=0.0000)   | 0.6589 (p=0.0000)    | 0.00%     |
| Modèle 2 (model)        | 0.6776 (p≈0.00)     | 0.6906 (p≈0.00)      | 56.66%    |
| Corpus Wikipedia (new\_model)   | 0.5434 (p≈0.00)     | 0.5427 (p≈0.00)      | 0.00%     |


> Le Modèle 2 (petit corpus) obtient les meilleures performances en similarité de mots avec des corrélations de Pearson (0,6776) et de Spearman (0,6906), mais il souffre d'un taux élevé de mots hors vocabulaire (OOV) de 56,66 %, ce qui limite son efficacité. Le Modèle wikipedia a une couverture complète du vocabulaire (0 % OOV) et un score d'analogie supérieur (0,3585) par rapport au Modèle 2 (0,3328), mais ses performances de similarité sont plus faibles (Pearson : 0,5434, Spearman : 0,5427). Le Modèle pré-entraîné est un compromis avec une couverture complète (0 % OOV) et des performances de similarité décentes (Pearson : 0,6239, Spearman : 0,6589). En résumé, le Modèle 2 (petit corpus) excelle en similarité mais manque de couverture, tandis que le corpus wikipedia combine couverture et bonnes performances en analogies, ce qui le rend plus polyvalent.